In [1]:
#Dependencies
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from scipy import stats 
import os
import gmaps
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy


In [2]:
#path to uber csv 

file="uber.csv"

#read into data
uber_df= pd.read_csv(file)
uber_df=uber_df.head(1000)


In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

pickup_lats=uber_df["pickup_latitude"]
pickup_lngs=uber_df["pickup_longitude"]
#lat_lngs=zip(pickup_lats,pickup_lngs)

uber_df["City"]="x"

for index, row in uber_df.iterrows():
    city=citipy.nearest_city(row["pickup_latitude"],row["pickup_longitude"]).city_name
    uber_df.loc[index,"City"]=city  
       
# Print the city count to confirm sufficient count
len(cities)
uber_df

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,City
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1,new york
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1,new york
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1,new york
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3,guttenberg
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5,new york
...,...,...,...,...,...,...,...,...,...,...
995,13439193,2011-05-04 06:39:00.00000044,5.7,2011-05-04 06:39:00 UTC,-73.969720,40.757577,-73.953782,40.766960,1,guttenberg
996,32405310,2011-11-23 20:43:20.0000002,8.1,2011-11-23 20:43:20 UTC,-73.993784,40.757054,-73.980018,40.775632,3,weehawken
997,51612001,2010-01-11 20:58:00.00000035,8.5,2010-01-11 20:58:00 UTC,-73.972338,40.765078,-73.954527,40.783833,5,guttenberg
998,937243,2013-06-12 17:01:24.0000001,5.5,2013-06-12 17:01:24 UTC,-73.979054,40.784730,-73.982970,40.775048,1,guttenberg


In [4]:
# vectorized haversine function
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))


In [5]:
#distance 

dropoff_lats=uber_df["pickup_longitude"]
dropoff_lngs=uber_df["pickup_longitude"]

uber_df["Distance"]=0

for index, row in uber_df.iterrows():
    distance=haversine(row["pickup_latitude"],row["pickup_longitude"],\
                       row["dropoff_latitude"], row["dropoff_longitude"],to_radians=True, earth_radius=6371)
    uber_df.loc[index,"Distance"]=distance  


In [6]:
#binning 

#set time of day bins
timeofday=[0, 5, 12, 17, 24]

uber_df.pickup_datetime= pd.to_datetime(uber_df.pickup_datetime)

#labels for bins
times=["Night","Morning","Afternoon","Evening"]

uber_df['Time of Day'] = pd.cut(uber_df.pickup_datetime.dt.hour, timeofday, labels=times, right=False)


In [7]:
#remove distance 0

uber_df_cleaned=uber_df.loc[uber_df["Distance"]!=0]

uber_df_cleaned

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,City,Distance,Time of Day
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,new york,1.683323,Evening
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,new york,2.457590,Evening
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,new york,5.036377,Evening
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,guttenberg,1.661683,Morning
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,new york,4.475450,Evening
...,...,...,...,...,...,...,...,...,...,...,...,...
995,13439193,2011-05-04 06:39:00.00000044,5.7,2011-05-04 06:39:00+00:00,-73.969720,40.757577,-73.953782,40.766960,1,guttenberg,1.700119,Morning
996,32405310,2011-11-23 20:43:20.0000002,8.1,2011-11-23 20:43:20+00:00,-73.993784,40.757054,-73.980018,40.775632,3,weehawken,2.368857,Evening
997,51612001,2010-01-11 20:58:00.00000035,8.5,2010-01-11 20:58:00+00:00,-73.972338,40.765078,-73.954527,40.783833,5,guttenberg,2.568764,Evening
998,937243,2013-06-12 17:01:24.0000001,5.5,2013-06-12 17:01:24+00:00,-73.979054,40.784730,-73.982970,40.775048,1,guttenberg,1.125950,Evening


In [8]:
# Most Popular Pick up times & how it correlates to price (plot, basic stats dataframe) -Neil 

# Rank by cities (fare amount paid, Total distances, Most popular time of day) -Rose 

# Where are the most popular pick up locations (map, dataframe) -Turgut 

# Price regression on number of passengers (plot, stats)- Neil 

# Where are the most popular drop off locations (map, dataframe) -Turgut

# Distance between pick up and drop off vs Price (basic stats, plot) -Daniel 

In [9]:
#sorting the datarame with City, Pickup Lats & Lngs, Dropoff Lats & Lngs

all_locations = uber_df_cleaned[['City', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']].groupby(uber_df_cleaned['City'])
all_locations.head()



,City,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,new york,40.738354,-73.999817,40.723217,-73.999512
1,new york,40.728225,-73.994355,40.750325,-73.994710
2,new york,40.740770,-74.005043,40.772647,-73.962565
3,guttenberg,40.790844,-73.976124,40.803349,-73.965316
4,new york,40.744085,-73.925023,40.761247,-73.973082
6,new york,40.693965,-73.961447,40.774297,-73.871195
12,edgewater,40.804440,-73.966378,40.807133,-73.965890
13,guttenberg,40.767382,-73.953352,40.796137,-73.972510
14,guttenberg,40.755193,-73.973370,40.766375,-73.978265
15,weehawken,40.751920,-73.990718,40.744230,-73.973053


In [10]:
#finding out the top cities
city_group = pd.DataFrame(uber_df_cleaned, columns=['City', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'])
top_cities = city_group[['City']].apply(pd.Series.value_counts).head(5)
top_cities


,City
new york,386
guttenberg,326
weehawken,155
edgewater,68
woodmere,12


In [11]:
#creating the dataframe to get the lats and lngs for pickup citites 
city_group = uber_df_cleaned.groupby("City")[["pickup_latitude", "pickup_longitude"]].mean().reset_index()

new_york = city_group.loc[city_group['City']=='new york']
guttenberg = city_group.loc[city_group['City']=='guttenberg']
weehawken = city_group.loc[city_group['City']=='weehawken']
edgewater = city_group.loc[city_group['City']=='edgewater']
woodmere = city_group.loc[city_group['City']=='woodmere']

top_pickups = pd.concat([new_york, guttenberg, weehawken, edgewater, woodmere], ignore_index=True, sort=True)
top_pickups

,City,pickup_latitude,pickup_longitude
0,new york,40.730186,-73.991053
1,guttenberg,40.767943,-73.971778
2,weehawken,40.753784,-73.990307
3,edgewater,40.790493,-73.942274
4,woodmere,40.644430,-73.783921


In [12]:
#creating the dataframe to get the lats and lngs for dropoff citites 

city_group = uber_df_cleaned.groupby("City")[["dropoff_latitude", "dropoff_longitude"]].mean().reset_index()

new_york = city_group.loc[city_group['City']=='new york']
guttenberg = city_group.loc[city_group['City']=='guttenberg']
weehawken = city_group.loc[city_group['City']=='weehawken']
edgewater = city_group.loc[city_group['City']=='edgewater']
woodmere = city_group.loc[city_group['City']=='woodmere']

top_dropoffs = pd.concat([new_york, guttenberg, weehawken, edgewater, woodmere], ignore_index=True, sort=True)
top_dropoffs


,City,dropoff_latitude,dropoff_longitude
0,new york,40.738688,-73.980338
1,guttenberg,40.760742,-73.970836
2,weehawken,40.750524,-73.977720
3,edgewater,40.773501,-73.959026
4,woodmere,40.720176,-73.953608


In [13]:

#setting pickup locations 
pickup_locations = top_pickups[["pickup_latitude", "pickup_longitude"]]
pickup_locations

#creating figure layout 
figure_layout = {'width': '1000px', 
                 'height': '500px', 
                 'border': '1px solid black', 
                 'padding': '0.01px', 
                 'margin': '0 auto 0 auto'}                       
fig = gmaps.figure(layout=figure_layout)

#adding markers 
markers = gmaps.marker_layer(pickup_locations[["pickup_latitude", "pickup_longitude"]])
fig.add_layer(markers)

#displaying the figure 
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='0.01px',…

In [14]:

#setting dropoff locations 
dropoff_locations = top_dropoffs[["dropoff_latitude", "dropoff_longitude"]]
dropoff_locations

#creating figure layout 
figure_layout = {'width': '1000px', 
                 'height': '500px', 
                 'border': '1px solid black', 
                 'padding': '0.01px', 
                 'margin': '0 auto 0 auto'}                        
fig = gmaps.figure(layout=figure_layout)
#adding markers 
markers = gmaps.marker_layer(dropoff_locations[["dropoff_latitude", "dropoff_longitude"]])
fig.add_layer(markers)

#dislpaying the figure 
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='0.01px',…